<a href="https://colab.research.google.com/github/ss-ghule/kaggle_predicting_molecular_properties/blob/master/c4_w2_feature_eng_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Aim: To create final training and test dataframes with new & important features
# Download data

In [0]:
# The code was removed by Watson Studio for sharing.

In [1]:
#!rm train_etl.csv test_etl.csv pairs_dict.pkl connectivity_model_nn_03.h5
!ls

champs-scalar-coupling_new.zip	    scalar_coupling_contributions.csv.zip
dipole_moments.csv.zip		    structures.csv.zip
kaggle.json			    structures.zip
magnetic_shielding_tensors.csv.zip  test.csv.zip
mulliken_charges.csv.zip	    test_etl.csv
potential_energy.csv.zip	    train.csv.zip
sample_data			    train_etl.csv
sample_submission.csv.zip


In [3]:
#download_file_cos(credentials_2, 'train_etl.csv', 'train_etl.csv')
#download_file_cos(credentials_2, 'test_etl.csv', 'test_etl.csv')
#download_file_cos(credentials_2, 'pairs_dict.pkl', 'pairs_dict.pkl')
#download_file_cos(credentials_2, 'connectivity_model_nn_03.h5', 'connectivity_model_nn_03.h5')
!ls

dipole_moments.csv.zip		    scalar_coupling_contributions.csv.zip
kaggle.json			    structures.csv.zip
magnetic_shielding_tensors.csv.zip  structures.zip
mulliken_charges.csv.zip	    test.csv.zip
potential_energy.csv.zip	    test_etl.csv
sample_data			    train.csv.zip
sample_submission.csv.zip	    train_etl.csv


In [0]:
#QM9
#!wget https://www.kaggleusercontent.com/kf/15612312/eyJhbGciOiJkaXIiLCJlbmMiOiJBMTI4Q0JDLUhTMjU2In0..XpvpY1AwxFHBalEO6EW75w.4xvQ52UpYKPAQp5lmOt_cSCeO_zDbr49lhqEr-LSneu9R5C_de3BtMl2KyXRR6uTUcwCzSfRb4vrmg5kFhzFjfg91HmxgfBTSuwrK3MBA62w4S8P51t083QEE4RXibSirX11HYn7eAph-MFMSHJ_yA.r8PDG6MhWtotvYsMKAnlnA/data.covs.pickle

In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle
import tensorflow as tf
from tensorflow import keras

%matplotlib inline

In [6]:
tf.__version__

'1.14.0'

In [45]:
train=pd.read_csv('train_etl.csv')
test=pd.read_csv('test_etl.csv')
print('train:{}'.format(train.shape))
print('test: {}'.format(test.shape))

train:(4658147, 13)
test: (2505542, 15)


In [8]:
train.head()

,id,molecule_name,atom_index_0,atom_index_1,atom_0,x_0,y_0,z_0,atom_1,x_1,y_1,z_1,mulliken_charge_0,mulliken_charge_1,type,scalar_coupling_constant
0,0,dsgdb9nsd_000001,1,0,H,0.002150,-0.006031,0.001976,C,-0.012698,1.085804,0.008001,0.133921,-0.535689,1JHC,84.8076
1,1,dsgdb9nsd_000001,1,2,H,0.002150,-0.006031,0.001976,H,1.011731,1.463751,0.000277,0.133921,0.133922,2JHH,-11.2570
2,2,dsgdb9nsd_000001,1,3,H,0.002150,-0.006031,0.001976,H,-0.540815,1.447527,-0.876644,0.133921,0.133923,2JHH,-11.2548
3,3,dsgdb9nsd_000001,1,4,H,0.002150,-0.006031,0.001976,H,-0.523814,1.437933,0.906397,0.133921,0.133923,2JHH,-11.2543
4,4,dsgdb9nsd_000001,2,0,H,1.011731,1.463751,0.000277,C,-0.012698,1.085804,0.008001,0.133922,-0.535689,1JHC,84.8074


In [9]:
test.head()

,id,molecule_name,atom_index_0,atom_index_1,atom_0,x_0,y_0,z_0,atom_1,x_1,y_1,z_1,mulliken_charge_0,mulliken_charge_1,type
0,4658147,dsgdb9nsd_000004,2,0,H,-1.661639,0.0,1.0,C,0.599539,0.0,1.0,NaN,NaN,2JHC
1,4658148,dsgdb9nsd_000004,2,1,H,-1.661639,0.0,1.0,C,-0.599539,0.0,1.0,NaN,NaN,1JHC
2,4658149,dsgdb9nsd_000004,2,3,H,-1.661639,0.0,1.0,H,1.661639,0.0,1.0,NaN,NaN,3JHH
3,4658150,dsgdb9nsd_000004,3,0,H,1.661639,0.0,1.0,C,0.599539,0.0,1.0,NaN,NaN,1JHC
4,4658151,dsgdb9nsd_000004,3,1,H,1.661639,0.0,1.0,C,-0.599539,0.0,1.0,NaN,NaN,2JHC


In [10]:
train.describe(include='all').T

,count,unique,top,freq,mean,std,min,25%,50%,75%,max
id,4.65815e+06,NaN,NaN,NaN,2.32907e+06,1.34469e+06,0,1.16454e+06,2.32907e+06,3.49361e+06,4.65815e+06
molecule_name,4658147,85003,dsgdb9nsd_042139,135,NaN,NaN,NaN,NaN,NaN,NaN,NaN
atom_index_0,4.65815e+06,NaN,NaN,NaN,13.3569,3.26771,0,11,13,16,28
atom_index_1,4.65815e+06,NaN,NaN,NaN,5.88397,4.99394,0,2,5,8,28
atom_0,4658147,1,H,4658147,NaN,NaN,NaN,NaN,NaN,NaN,NaN
x_0,4.65815e+06,NaN,NaN,NaN,0.105878,1.75987,-9.23489,-0.953092,0.134121,1.22775,9.38224
y_0,4.65815e+06,NaN,NaN,NaN,-0.220369,2.12179,-9.49416,-1.83697,-0.297994,1.75342,9.71447
z_0,4.65815e+06,NaN,NaN,NaN,0.0471289,1.56451,-9.13476,-0.964213,0.00497289,1.0351,7.63758
atom_1,4658147,3,C,3360469,NaN,NaN,NaN,NaN,NaN,NaN,NaN
x_1,4.65815e+06,NaN,NaN,NaN,0.0975718,1.47942,-9.23489,-0.79276,0.0466042,0.968224,8.42089


In [11]:
test.describe(include='all').T

,count,unique,top,freq,mean,std,min,25%,50%,75%,max
id,2.50554e+06,NaN,NaN,NaN,5.91092e+06,723288,4.65815e+06,5.28453e+06,5.91092e+06,6.5373e+06,7.16369e+06
molecule_name,2505542,45772,dsgdb9nsd_041946,131,NaN,NaN,NaN,NaN,NaN,NaN,NaN
atom_index_0,2.50554e+06,NaN,NaN,NaN,13.3545,3.27221,1,11,13,16,28
atom_index_1,2.50554e+06,NaN,NaN,NaN,5.87795,4.99231,0,2,5,8,28
atom_0,2505542,1,H,2505542,NaN,NaN,NaN,NaN,NaN,NaN,NaN
x_0,2.50554e+06,NaN,NaN,NaN,0.111877,1.75132,-8.35577,-0.946371,0.139294,1.22997,9.15701
y_0,2.50554e+06,NaN,NaN,NaN,-0.223317,2.1128,-9.93394,-1.83265,-0.300268,1.74771,10.182
z_0,2.50554e+06,NaN,NaN,NaN,0.0532973,1.56707,-7.82342,-0.960673,0.00780067,1.04035,7.89473
atom_1,2505542,3,C,1805746,NaN,NaN,NaN,NaN,NaN,NaN,NaN
x_1,2.50554e+06,NaN,NaN,NaN,0.102163,1.4705,-8.35577,-0.775903,0.0472481,0.968822,8.18919


# Feature Engineering (train)

## Euclidian Distance

In [0]:
def get_distance(df):
    return np.sqrt(np.square(df['x_0']-df['x_1'])+np.square(df['y_0']-df['y_1'])+np.square(df['z_0']-df['z_1']))

In [13]:
train['distance']=train.apply(get_distance,axis=1)
train.head()

,id,molecule_name,atom_index_0,atom_index_1,atom_0,x_0,y_0,z_0,atom_1,x_1,y_1,z_1,mulliken_charge_0,mulliken_charge_1,type,scalar_coupling_constant,distance
0,0,dsgdb9nsd_000001,1,0,H,0.002150,-0.006031,0.001976,C,-0.012698,1.085804,0.008001,0.133921,-0.535689,1JHC,84.8076,1.091953
1,1,dsgdb9nsd_000001,1,2,H,0.002150,-0.006031,0.001976,H,1.011731,1.463751,0.000277,0.133921,0.133922,2JHH,-11.2570,1.783120
2,2,dsgdb9nsd_000001,1,3,H,0.002150,-0.006031,0.001976,H,-0.540815,1.447527,-0.876644,0.133921,0.133923,2JHH,-11.2548,1.783147
3,3,dsgdb9nsd_000001,1,4,H,0.002150,-0.006031,0.001976,H,-0.523814,1.437933,0.906397,0.133921,0.133923,2JHH,-11.2543,1.783157
4,4,dsgdb9nsd_000001,2,0,H,1.011731,1.463751,0.000277,C,-0.012698,1.085804,0.008001,0.133922,-0.535689,1JHC,84.8074,1.091952


## one-hot  encode atom names
(Not neccessary)

In [0]:
#train=train.join(pd.get_dummies(train['atom_0'],prefix='atom_0'))
#train.head()

In [0]:
#train=train.join(pd.get_dummies(train['atom_1'],prefix='atom_1'))
#train.head()

## Label Encode type

In [0]:
from sklearn.preprocessing import LabelEncoder
le_type=LabelEncoder()
train['type_enc']=le_type.fit_transform(train['type'])

In [15]:
train.head()

,id,molecule_name,atom_index_0,atom_index_1,atom_0,x_0,y_0,z_0,atom_1,x_1,y_1,z_1,mulliken_charge_0,mulliken_charge_1,type,scalar_coupling_constant,distance,type_enc
0,0,dsgdb9nsd_000001,1,0,H,0.002150,-0.006031,0.001976,C,-0.012698,1.085804,0.008001,0.133921,-0.535689,1JHC,84.8076,1.091953,0
1,1,dsgdb9nsd_000001,1,2,H,0.002150,-0.006031,0.001976,H,1.011731,1.463751,0.000277,0.133921,0.133922,2JHH,-11.2570,1.783120,3
2,2,dsgdb9nsd_000001,1,3,H,0.002150,-0.006031,0.001976,H,-0.540815,1.447527,-0.876644,0.133921,0.133923,2JHH,-11.2548,1.783147,3
3,3,dsgdb9nsd_000001,1,4,H,0.002150,-0.006031,0.001976,H,-0.523814,1.437933,0.906397,0.133921,0.133923,2JHH,-11.2543,1.783157,3
4,4,dsgdb9nsd_000001,2,0,H,1.011731,1.463751,0.000277,C,-0.012698,1.085804,0.008001,0.133922,-0.535689,1JHC,84.8074,1.091952,0


## Label Encode Atoms

In [0]:
le_atoms=LabelEncoder()
train['atom_1_enc']=le_atoms.fit_transform(train['atom_1'])
train['atom_0_enc']=le_atoms.transform(train['atom_0'])

In [17]:
train.head()

,id,molecule_name,atom_index_0,atom_index_1,atom_0,x_0,y_0,z_0,atom_1,x_1,y_1,z_1,mulliken_charge_0,mulliken_charge_1,type,scalar_coupling_constant,distance,type_enc,atom_1_enc,atom_0_enc
0,0,dsgdb9nsd_000001,1,0,H,0.002150,-0.006031,0.001976,C,-0.012698,1.085804,0.008001,0.133921,-0.535689,1JHC,84.8076,1.091953,0,0,1
1,1,dsgdb9nsd_000001,1,2,H,0.002150,-0.006031,0.001976,H,1.011731,1.463751,0.000277,0.133921,0.133922,2JHH,-11.2570,1.783120,3,1,1
2,2,dsgdb9nsd_000001,1,3,H,0.002150,-0.006031,0.001976,H,-0.540815,1.447527,-0.876644,0.133921,0.133923,2JHH,-11.2548,1.783147,3,1,1
3,3,dsgdb9nsd_000001,1,4,H,0.002150,-0.006031,0.001976,H,-0.523814,1.437933,0.906397,0.133921,0.133923,2JHH,-11.2543,1.783157,3,1,1
4,4,dsgdb9nsd_000001,2,0,H,1.011731,1.463751,0.000277,C,-0.012698,1.085804,0.008001,0.133922,-0.535689,1JHC,84.8074,1.091952,0,0,1


## Pairs

In [0]:
train['pairs']=(train['atom_0']+'_'+train['atom_1']).apply(lambda x:x.split('_')).apply(sorted).apply(lambda x:'_'.join(x)).apply(str.lower)

In [0]:
train.head()

,id,molecule_name,atom_index_0,atom_index_1,atom_0,x_0,y_0,z_0,atom_1,x_1,y_1,z_1,mulliken_charge_0,mulliken_charge_1,type,scalar_coupling_constant,distance,type_enc,pairs
0,0,dsgdb9nsd_000001,1,0,H,0.002150,-0.006031,0.001976,C,-0.012698,1.085804,0.008001,0.133921,-0.535689,1JHC,84.8076,1.091953,0,c_h
1,1,dsgdb9nsd_000001,1,2,H,0.002150,-0.006031,0.001976,H,1.011731,1.463751,0.000277,0.133921,0.133922,2JHH,-11.2570,1.783120,3,h_h
2,2,dsgdb9nsd_000001,1,3,H,0.002150,-0.006031,0.001976,H,-0.540815,1.447527,-0.876644,0.133921,0.133923,2JHH,-11.2548,1.783147,3,h_h
3,3,dsgdb9nsd_000001,1,4,H,0.002150,-0.006031,0.001976,H,-0.523814,1.437933,0.906397,0.133921,0.133923,2JHH,-11.2543,1.783157,3,h_h
4,4,dsgdb9nsd_000001,2,0,H,1.011731,1.463751,0.000277,C,-0.012698,1.085804,0.008001,0.133922,-0.535689,1JHC,84.8074,1.091952,0,c_h


,id,molecule_name,atom_index_0,atom_index_1,atom_0,x_0,y_0,z_0,atom_1,x_1,y_1,z_1,mulliken_charge_0,mulliken_charge_1,type,scalar_coupling_constant,distance,type_enc,pairs
0,0,dsgdb9nsd_000001,1,0,H,0.002150,-0.006031,0.001976,C,-0.012698,1.085804,0.008001,0.133921,-0.535689,1JHC,84.8076,1.091953,0,c_h
1,1,dsgdb9nsd_000001,1,2,H,0.002150,-0.006031,0.001976,H,1.011731,1.463751,0.000277,0.133921,0.133922,2JHH,-11.2570,1.783120,3,h_h
2,2,dsgdb9nsd_000001,1,3,H,0.002150,-0.006031,0.001976,H,-0.540815,1.447527,-0.876644,0.133921,0.133923,2JHH,-11.2548,1.783147,3,h_h
3,3,dsgdb9nsd_000001,1,4,H,0.002150,-0.006031,0.001976,H,-0.523814,1.437933,0.906397,0.133921,0.133923,2JHH,-11.2543,1.783157,3,h_h
4,4,dsgdb9nsd_000001,2,0,H,1.011731,1.463751,0.000277,C,-0.012698,1.085804,0.008001,0.133922,-0.535689,1JHC,84.8074,1.091952,0,c_h


## Label Encode Pairs using dictionary

In [0]:
!ls

connectivity_model_nn_03.h5  pairs_dict.pkl  test_etl.csv  train_etl.csv
connectivity_model_nn_03.h5  pairs_dict.pkl  test_etl.csv  train_etl.csv


In [0]:
with open('pairs_dict.pkl','rb') as f:
    pairs_dict=pickle.load(f)

In [0]:
pairs_dict

{'c_c': 0,
 'c_f': 1,
 'c_h': 2,
 'c_n': 3,
 'c_o': 4,
 'c_si': 5,
 'f_f': 6,
 'f_h': 7,
 'f_n': 8,
 'f_o': 9,
 'f_si': 10,
 'h_h': 11,
 'h_n': 12,
 'h_o': 13,
 'h_si': 14,
 'n_n': 15,
 'n_o': 16,
 'o_o': 17}

{'c_c': 0,
 'c_f': 1,
 'c_h': 2,
 'c_n': 3,
 'c_o': 4,
 'c_si': 5,
 'f_f': 6,
 'f_h': 7,
 'f_n': 8,
 'f_o': 9,
 'f_si': 10,
 'h_h': 11,
 'h_n': 12,
 'h_o': 13,
 'h_si': 14,
 'n_n': 15,
 'n_o': 16,
 'o_o': 17}

In [0]:
train['pairs'].unique()

array(['c_h', 'h_h', 'h_n'], dtype=object)

array(['c_h', 'h_h', 'h_n'], dtype=object)

In [0]:
train['pairs_enc']=train['pairs'].map(pairs_dict)

In [0]:
train.head()

,id,molecule_name,atom_index_0,atom_index_1,atom_0,x_0,y_0,z_0,atom_1,x_1,y_1,z_1,mulliken_charge_0,mulliken_charge_1,type,scalar_coupling_constant,distance,type_enc,pairs,pairs_enc
0,0,dsgdb9nsd_000001,1,0,H,0.002150,-0.006031,0.001976,C,-0.012698,1.085804,0.008001,0.133921,-0.535689,1JHC,84.8076,1.091953,0,c_h,2
1,1,dsgdb9nsd_000001,1,2,H,0.002150,-0.006031,0.001976,H,1.011731,1.463751,0.000277,0.133921,0.133922,2JHH,-11.2570,1.783120,3,h_h,11
2,2,dsgdb9nsd_000001,1,3,H,0.002150,-0.006031,0.001976,H,-0.540815,1.447527,-0.876644,0.133921,0.133923,2JHH,-11.2548,1.783147,3,h_h,11
3,3,dsgdb9nsd_000001,1,4,H,0.002150,-0.006031,0.001976,H,-0.523814,1.437933,0.906397,0.133921,0.133923,2JHH,-11.2543,1.783157,3,h_h,11
4,4,dsgdb9nsd_000001,2,0,H,1.011731,1.463751,0.000277,C,-0.012698,1.085804,0.008001,0.133922,-0.535689,1JHC,84.8074,1.091952,0,c_h,2


,id,molecule_name,atom_index_0,atom_index_1,atom_0,x_0,y_0,z_0,atom_1,x_1,y_1,z_1,mulliken_charge_0,mulliken_charge_1,type,scalar_coupling_constant,distance,type_enc,pairs,pairs_enc
0,0,dsgdb9nsd_000001,1,0,H,0.002150,-0.006031,0.001976,C,-0.012698,1.085804,0.008001,0.133921,-0.535689,1JHC,84.8076,1.091953,0,c_h,2
1,1,dsgdb9nsd_000001,1,2,H,0.002150,-0.006031,0.001976,H,1.011731,1.463751,0.000277,0.133921,0.133922,2JHH,-11.2570,1.783120,3,h_h,11
2,2,dsgdb9nsd_000001,1,3,H,0.002150,-0.006031,0.001976,H,-0.540815,1.447527,-0.876644,0.133921,0.133923,2JHH,-11.2548,1.783147,3,h_h,11
3,3,dsgdb9nsd_000001,1,4,H,0.002150,-0.006031,0.001976,H,-0.523814,1.437933,0.906397,0.133921,0.133923,2JHH,-11.2543,1.783157,3,h_h,11
4,4,dsgdb9nsd_000001,2,0,H,1.011731,1.463751,0.000277,C,-0.012698,1.085804,0.008001,0.133922,-0.535689,1JHC,84.8074,1.091952,0,c_h,2


## Connectivity

In [0]:
model_nn=tf.keras.models.load_model('connectivity_model_nn_03.h5',custom_objects=None,compile=True)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Use tf.cast instead.


In [0]:
x=train[['pairs_enc','distance']]

In [0]:
x.head()

,pairs_enc,distance
0,2,1.091953
1,11,1.783120
2,11,1.783147
3,11,1.783157
4,2,1.091952


,pairs_enc,distance
0,2,1.091953
1,11,1.783120
2,11,1.783147
3,11,1.783157
4,2,1.091952


In [0]:
x.shape

(4658147, 2)

(4658147, 2)

In [0]:
#model_nn.predict(x,verbose=1)

In [0]:
train['bond']=model_nn.predict(x,verbose=1,batch_size=10000)

4658147/4658147 [==============================] - 93s 20us/sample


In [0]:
train.head()

,id,molecule_name,atom_index_0,atom_index_1,atom_0,x_0,y_0,z_0,atom_1,x_1,...,z_1,mulliken_charge_0,mulliken_charge_1,type,scalar_coupling_constant,distance,type_enc,pairs,pairs_enc,bond
0,0,dsgdb9nsd_000001,1,0,H,0.002150,-0.006031,0.001976,C,-0.012698,...,0.008001,0.133921,-0.535689,1JHC,84.8076,1.091953,0,c_h,2,0.919760
1,1,dsgdb9nsd_000001,1,2,H,0.002150,-0.006031,0.001976,H,1.011731,...,0.000277,0.133921,0.133922,2JHH,-11.2570,1.783120,3,h_h,11,0.648310
2,2,dsgdb9nsd_000001,1,3,H,0.002150,-0.006031,0.001976,H,-0.540815,...,-0.876644,0.133921,0.133923,2JHH,-11.2548,1.783147,3,h_h,11,0.648205
3,3,dsgdb9nsd_000001,1,4,H,0.002150,-0.006031,0.001976,H,-0.523814,...,0.906397,0.133921,0.133923,2JHH,-11.2543,1.783157,3,h_h,11,0.648170
4,4,dsgdb9nsd_000001,2,0,H,1.011731,1.463751,0.000277,C,-0.012698,...,0.008001,0.133922,-0.535689,1JHC,84.8074,1.091952,0,c_h,2,0.919760


,id,molecule_name,atom_index_0,atom_index_1,atom_0,x_0,y_0,z_0,atom_1,x_1,...,z_1,mulliken_charge_0,mulliken_charge_1,type,scalar_coupling_constant,distance,type_enc,pairs,pairs_enc,bond
0,0,dsgdb9nsd_000001,1,0,H,0.002150,-0.006031,0.001976,C,-0.012698,...,0.008001,0.133921,-0.535689,1JHC,84.8076,1.091953,0,c_h,2,0.919760
1,1,dsgdb9nsd_000001,1,2,H,0.002150,-0.006031,0.001976,H,1.011731,...,0.000277,0.133921,0.133922,2JHH,-11.2570,1.783120,3,h_h,11,0.648310
2,2,dsgdb9nsd_000001,1,3,H,0.002150,-0.006031,0.001976,H,-0.540815,...,-0.876644,0.133921,0.133923,2JHH,-11.2548,1.783147,3,h_h,11,0.648205
3,3,dsgdb9nsd_000001,1,4,H,0.002150,-0.006031,0.001976,H,-0.523814,...,0.906397,0.133921,0.133923,2JHH,-11.2543,1.783157,3,h_h,11,0.648170
4,4,dsgdb9nsd_000001,2,0,H,1.011731,1.463751,0.000277,C,-0.012698,...,0.008001,0.133922,-0.535689,1JHC,84.8074,1.091952,0,c_h,2,0.919760


In [0]:
train['bond'].isna().value_counts()

False    4658147
Name: bond, dtype: int64

False    4658147
Name: bond, dtype: int64

In [0]:
train['bond_int']=train['bond'].apply(lambda x:int(x>0.5))

In [0]:
train.head()

,id,molecule_name,atom_index_0,atom_index_1,atom_0,x_0,y_0,z_0,atom_1,x_1,...,mulliken_charge_0,mulliken_charge_1,type,scalar_coupling_constant,distance,type_enc,pairs,pairs_enc,bond,bond_int
0,0,dsgdb9nsd_000001,1,0,H,0.002150,-0.006031,0.001976,C,-0.012698,...,0.133921,-0.535689,1JHC,84.8076,1.091953,0,c_h,2,0.919760,1
1,1,dsgdb9nsd_000001,1,2,H,0.002150,-0.006031,0.001976,H,1.011731,...,0.133921,0.133922,2JHH,-11.2570,1.783120,3,h_h,11,0.648310,1
2,2,dsgdb9nsd_000001,1,3,H,0.002150,-0.006031,0.001976,H,-0.540815,...,0.133921,0.133923,2JHH,-11.2548,1.783147,3,h_h,11,0.648205,1
3,3,dsgdb9nsd_000001,1,4,H,0.002150,-0.006031,0.001976,H,-0.523814,...,0.133921,0.133923,2JHH,-11.2543,1.783157,3,h_h,11,0.648170,1
4,4,dsgdb9nsd_000001,2,0,H,1.011731,1.463751,0.000277,C,-0.012698,...,0.133922,-0.535689,1JHC,84.8074,1.091952,0,c_h,2,0.919760,1


,id,molecule_name,atom_index_0,atom_index_1,atom_0,x_0,y_0,z_0,atom_1,x_1,...,mulliken_charge_0,mulliken_charge_1,type,scalar_coupling_constant,distance,type_enc,pairs,pairs_enc,bond,bond_int
0,0,dsgdb9nsd_000001,1,0,H,0.002150,-0.006031,0.001976,C,-0.012698,...,0.133921,-0.535689,1JHC,84.8076,1.091953,0,c_h,2,0.919760,1
1,1,dsgdb9nsd_000001,1,2,H,0.002150,-0.006031,0.001976,H,1.011731,...,0.133921,0.133922,2JHH,-11.2570,1.783120,3,h_h,11,0.648310,1
2,2,dsgdb9nsd_000001,1,3,H,0.002150,-0.006031,0.001976,H,-0.540815,...,0.133921,0.133923,2JHH,-11.2548,1.783147,3,h_h,11,0.648205,1
3,3,dsgdb9nsd_000001,1,4,H,0.002150,-0.006031,0.001976,H,-0.523814,...,0.133921,0.133923,2JHH,-11.2543,1.783157,3,h_h,11,0.648170,1
4,4,dsgdb9nsd_000001,2,0,H,1.011731,1.463751,0.000277,C,-0.012698,...,0.133922,-0.535689,1JHC,84.8074,1.091952,0,c_h,2,0.919760,1


## Total Charge

In [0]:
train['total_charge']=train['mulliken_charge_0']+train['mulliken_charge_1']

In [0]:
train.head()

,id,molecule_name,atom_index_0,atom_index_1,atom_0,x_0,y_0,z_0,atom_1,x_1,...,mulliken_charge_1,type,scalar_coupling_constant,distance,type_enc,pairs,pairs_enc,bond,bond_int,total_charge
0,0,dsgdb9nsd_000001,1,0,H,0.002150,-0.006031,0.001976,C,-0.012698,...,-0.535689,1JHC,84.8076,1.091953,0,c_h,2,0.919760,1,-0.401768
1,1,dsgdb9nsd_000001,1,2,H,0.002150,-0.006031,0.001976,H,1.011731,...,0.133922,2JHH,-11.2570,1.783120,3,h_h,11,0.648310,1,0.267843
2,2,dsgdb9nsd_000001,1,3,H,0.002150,-0.006031,0.001976,H,-0.540815,...,0.133923,2JHH,-11.2548,1.783147,3,h_h,11,0.648205,1,0.267844
3,3,dsgdb9nsd_000001,1,4,H,0.002150,-0.006031,0.001976,H,-0.523814,...,0.133923,2JHH,-11.2543,1.783157,3,h_h,11,0.648170,1,0.267844
4,4,dsgdb9nsd_000001,2,0,H,1.011731,1.463751,0.000277,C,-0.012698,...,-0.535689,1JHC,84.8074,1.091952,0,c_h,2,0.919760,1,-0.401767


,id,molecule_name,atom_index_0,atom_index_1,atom_0,x_0,y_0,z_0,atom_1,x_1,...,mulliken_charge_1,type,scalar_coupling_constant,distance,type_enc,pairs,pairs_enc,bond,bond_int,total_charge
0,0,dsgdb9nsd_000001,1,0,H,0.002150,-0.006031,0.001976,C,-0.012698,...,-0.535689,1JHC,84.8076,1.091953,0,c_h,2,0.919760,1,-0.401768
1,1,dsgdb9nsd_000001,1,2,H,0.002150,-0.006031,0.001976,H,1.011731,...,0.133922,2JHH,-11.2570,1.783120,3,h_h,11,0.648310,1,0.267843
2,2,dsgdb9nsd_000001,1,3,H,0.002150,-0.006031,0.001976,H,-0.540815,...,0.133923,2JHH,-11.2548,1.783147,3,h_h,11,0.648205,1,0.267844
3,3,dsgdb9nsd_000001,1,4,H,0.002150,-0.006031,0.001976,H,-0.523814,...,0.133923,2JHH,-11.2543,1.783157,3,h_h,11,0.648170,1,0.267844
4,4,dsgdb9nsd_000001,2,0,H,1.011731,1.463751,0.000277,C,-0.012698,...,-0.535689,1JHC,84.8074,1.091952,0,c_h,2,0.919760,1,-0.401767


## Dipole moment

sum(qi*ri)

In [0]:
train['dp']=(-1*train['mulliken_charge_0']*train['distance']/2)+(train['mulliken_charge_1']*train['distance']/2)

In [0]:
train.head()

,id,molecule_name,atom_index_0,atom_index_1,atom_0,x_0,y_0,z_0,atom_1,x_1,...,type,scalar_coupling_constant,distance,type_enc,pairs,pairs_enc,bond,bond_int,total_charge,dp
0,0,dsgdb9nsd_000001,1,0,H,0.002150,-0.006031,0.001976,C,-0.012698,...,1JHC,84.8076,1.091953,0,c_h,2,0.919760,1,-0.401768,-3.655913e-01
1,1,dsgdb9nsd_000001,1,2,H,0.002150,-0.006031,0.001976,H,1.011731,...,2JHH,-11.2570,1.783120,3,h_h,11,0.648310,1,0.267843,8.915599e-07
2,2,dsgdb9nsd_000001,1,3,H,0.002150,-0.006031,0.001976,H,-0.540815,...,2JHH,-11.2548,1.783147,3,h_h,11,0.648205,1,0.267844,1.783147e-06
3,3,dsgdb9nsd_000001,1,4,H,0.002150,-0.006031,0.001976,H,-0.523814,...,2JHH,-11.2543,1.783157,3,h_h,11,0.648170,1,0.267844,1.783157e-06
4,4,dsgdb9nsd_000001,2,0,H,1.011731,1.463751,0.000277,C,-0.012698,...,1JHC,84.8074,1.091952,0,c_h,2,0.919760,1,-0.401767,-3.655914e-01


,id,molecule_name,atom_index_0,atom_index_1,atom_0,x_0,y_0,z_0,atom_1,x_1,...,type,scalar_coupling_constant,distance,type_enc,pairs,pairs_enc,bond,bond_int,total_charge,dp
0,0,dsgdb9nsd_000001,1,0,H,0.002150,-0.006031,0.001976,C,-0.012698,...,1JHC,84.8076,1.091953,0,c_h,2,0.919760,1,-0.401768,-3.655913e-01
1,1,dsgdb9nsd_000001,1,2,H,0.002150,-0.006031,0.001976,H,1.011731,...,2JHH,-11.2570,1.783120,3,h_h,11,0.648310,1,0.267843,8.915599e-07
2,2,dsgdb9nsd_000001,1,3,H,0.002150,-0.006031,0.001976,H,-0.540815,...,2JHH,-11.2548,1.783147,3,h_h,11,0.648205,1,0.267844,1.783147e-06
3,3,dsgdb9nsd_000001,1,4,H,0.002150,-0.006031,0.001976,H,-0.523814,...,2JHH,-11.2543,1.783157,3,h_h,11,0.648170,1,0.267844,1.783157e-06
4,4,dsgdb9nsd_000001,2,0,H,1.011731,1.463751,0.000277,C,-0.012698,...,1JHC,84.8074,1.091952,0,c_h,2,0.919760,1,-0.401767,-3.655914e-01


## Bond angle, dihedral angle

# Feature Engineering (test)

## Mulliken Charge (Imputation)

In [14]:
test.head()

,id,molecule_name,atom_index_0,atom_index_1,atom_0,x_0,y_0,z_0,atom_1,x_1,y_1,z_1,mulliken_charge_0,mulliken_charge_1,type
0,4658147,dsgdb9nsd_000004,2,0,H,-1.661639,0.0,1.0,C,0.599539,0.0,1.0,NaN,NaN,2JHC
1,4658148,dsgdb9nsd_000004,2,1,H,-1.661639,0.0,1.0,C,-0.599539,0.0,1.0,NaN,NaN,1JHC
2,4658149,dsgdb9nsd_000004,2,3,H,-1.661639,0.0,1.0,H,1.661639,0.0,1.0,NaN,NaN,3JHH
3,4658150,dsgdb9nsd_000004,3,0,H,1.661639,0.0,1.0,C,0.599539,0.0,1.0,NaN,NaN,1JHC
4,4658151,dsgdb9nsd_000004,3,1,H,1.661639,0.0,1.0,C,-0.599539,0.0,1.0,NaN,NaN,2JHC


In [15]:
avg_charge=pd.DataFrame(train.groupby('atom_1')['mulliken_charge_1'].mean()).reset_index()
avg_charge.head()

,atom_1,mulliken_charge_1
0,C,-0.064872
1,H,0.118187
2,N,-0.269914


In [16]:
test.drop('mulliken_charge_1',axis=1,inplace=True)
test.head()

,id,molecule_name,atom_index_0,atom_index_1,atom_0,x_0,y_0,z_0,atom_1,x_1,y_1,z_1,mulliken_charge_0,type
0,4658147,dsgdb9nsd_000004,2,0,H,-1.661639,0.0,1.0,C,0.599539,0.0,1.0,NaN,2JHC
1,4658148,dsgdb9nsd_000004,2,1,H,-1.661639,0.0,1.0,C,-0.599539,0.0,1.0,NaN,1JHC
2,4658149,dsgdb9nsd_000004,2,3,H,-1.661639,0.0,1.0,H,1.661639,0.0,1.0,NaN,3JHH
3,4658150,dsgdb9nsd_000004,3,0,H,1.661639,0.0,1.0,C,0.599539,0.0,1.0,NaN,1JHC
4,4658151,dsgdb9nsd_000004,3,1,H,1.661639,0.0,1.0,C,-0.599539,0.0,1.0,NaN,2JHC


In [17]:
test=pd.merge(test,avg_charge,on='atom_1',how='left')
test.head()

,id,molecule_name,atom_index_0,atom_index_1,atom_0,x_0,y_0,z_0,atom_1,x_1,y_1,z_1,mulliken_charge_0,type,mulliken_charge_1
0,4658147,dsgdb9nsd_000004,2,0,H,-1.661639,0.0,1.0,C,0.599539,0.0,1.0,NaN,2JHC,-0.064872
1,4658148,dsgdb9nsd_000004,2,1,H,-1.661639,0.0,1.0,C,-0.599539,0.0,1.0,NaN,1JHC,-0.064872
2,4658149,dsgdb9nsd_000004,2,3,H,-1.661639,0.0,1.0,H,1.661639,0.0,1.0,NaN,3JHH,0.118187
3,4658150,dsgdb9nsd_000004,3,0,H,1.661639,0.0,1.0,C,0.599539,0.0,1.0,NaN,1JHC,-0.064872
4,4658151,dsgdb9nsd_000004,3,1,H,1.661639,0.0,1.0,C,-0.599539,0.0,1.0,NaN,2JHC,-0.064872


In [45]:
#only H is present
test['mulliken_charge_0']=train['mulliken_charge_0'].mean()
test.head()

,atom_index_0,atom_index_1,x_0,y_0,z_0,x_1,y_1,z_1,mulliken_charge_0,mulliken_charge_1,distance,type_enc
0,2,0,-1.661639,0.0,1.0,0.599539,0.0,1.0,0.12211,-0.064872,2.261178,2
1,2,1,-1.661639,0.0,1.0,-0.599539,0.0,1.0,0.12211,-0.064872,1.062099,0
2,2,3,-1.661639,0.0,1.0,1.661639,0.0,1.0,0.12211,0.118187,3.323277,6
3,3,0,1.661639,0.0,1.0,0.599539,0.0,1.0,0.12211,-0.064872,1.062099,0
4,3,1,1.661639,0.0,1.0,-0.599539,0.0,1.0,0.12211,-0.064872,2.261178,2


## Euclidian Distance

In [46]:
test['distance']=get_distance(test)
test.head()

,id,molecule_name,atom_index_0,atom_index_1,atom_0,x_0,y_0,z_0,atom_1,x_1,y_1,z_1,mulliken_charge_0,mulliken_charge_1,type,distance
0,4658147,dsgdb9nsd_000004,2,0,H,-1.661639,0.0,1.0,C,0.599539,0.0,1.0,NaN,NaN,2JHC,2.261178
1,4658148,dsgdb9nsd_000004,2,1,H,-1.661639,0.0,1.0,C,-0.599539,0.0,1.0,NaN,NaN,1JHC,1.062099
2,4658149,dsgdb9nsd_000004,2,3,H,-1.661639,0.0,1.0,H,1.661639,0.0,1.0,NaN,NaN,3JHH,3.323277
3,4658150,dsgdb9nsd_000004,3,0,H,1.661639,0.0,1.0,C,0.599539,0.0,1.0,NaN,NaN,1JHC,1.062099
4,4658151,dsgdb9nsd_000004,3,1,H,1.661639,0.0,1.0,C,-0.599539,0.0,1.0,NaN,NaN,2JHC,2.261178


## one-hot encode atom names
(Not neccessary)

In [0]:
#test=test.join(pd.get_dummies(test['atom_0'],prefix='atom_0'))
#test.head()

In [0]:
#test=test.join(pd.get_dummies(test['atom_1'],prefix='atom_1'))
#test.head()

## Label Encode type

In [47]:
test['type_enc']=le_type.transform(test['type'])
test.head()

,id,molecule_name,atom_index_0,atom_index_1,atom_0,x_0,y_0,z_0,atom_1,x_1,y_1,z_1,mulliken_charge_0,mulliken_charge_1,type,distance,type_enc
0,4658147,dsgdb9nsd_000004,2,0,H,-1.661639,0.0,1.0,C,0.599539,0.0,1.0,NaN,NaN,2JHC,2.261178,2
1,4658148,dsgdb9nsd_000004,2,1,H,-1.661639,0.0,1.0,C,-0.599539,0.0,1.0,NaN,NaN,1JHC,1.062099,0
2,4658149,dsgdb9nsd_000004,2,3,H,-1.661639,0.0,1.0,H,1.661639,0.0,1.0,NaN,NaN,3JHH,3.323277,6
3,4658150,dsgdb9nsd_000004,3,0,H,1.661639,0.0,1.0,C,0.599539,0.0,1.0,NaN,NaN,1JHC,1.062099,0
4,4658151,dsgdb9nsd_000004,3,1,H,1.661639,0.0,1.0,C,-0.599539,0.0,1.0,NaN,NaN,2JHC,2.261178,2


## Label Encode Atoms

In [48]:
test['atom_0_enc']=le_atoms.transform(test['atom_0'])
test['atom_1_enc']=le_atoms.transform(test['atom_1'])
test.head()

,id,molecule_name,atom_index_0,atom_index_1,atom_0,x_0,y_0,z_0,atom_1,x_1,y_1,z_1,mulliken_charge_0,mulliken_charge_1,type,distance,type_enc,atom_0_enc,atom_1_enc
0,4658147,dsgdb9nsd_000004,2,0,H,-1.661639,0.0,1.0,C,0.599539,0.0,1.0,NaN,NaN,2JHC,2.261178,2,1,0
1,4658148,dsgdb9nsd_000004,2,1,H,-1.661639,0.0,1.0,C,-0.599539,0.0,1.0,NaN,NaN,1JHC,1.062099,0,1,0
2,4658149,dsgdb9nsd_000004,2,3,H,-1.661639,0.0,1.0,H,1.661639,0.0,1.0,NaN,NaN,3JHH,3.323277,6,1,1
3,4658150,dsgdb9nsd_000004,3,0,H,1.661639,0.0,1.0,C,0.599539,0.0,1.0,NaN,NaN,1JHC,1.062099,0,1,0
4,4658151,dsgdb9nsd_000004,3,1,H,1.661639,0.0,1.0,C,-0.599539,0.0,1.0,NaN,NaN,2JHC,2.261178,2,1,0


## Pairs

In [0]:
test['pairs']=(test['atom_0']+'_'+test['atom_1']).apply(lambda x:x.split('_')).apply(sorted).apply(lambda x:'_'.join(x)).apply(str.lower)

In [0]:
test.head()

,id,molecule_name,atom_index_0,atom_index_1,atom_0,x_0,y_0,z_0,atom_1,x_1,y_1,z_1,mulliken_charge_0,type,mulliken_charge_1,distance,type_enc,pairs
0,4658147,dsgdb9nsd_000004,2,0,H,-1.661639,0.0,1.0,C,0.599539,0.0,1.0,0.12211,2JHC,-0.064872,2.261178,2,c_h
1,4658148,dsgdb9nsd_000004,2,1,H,-1.661639,0.0,1.0,C,-0.599539,0.0,1.0,0.12211,1JHC,-0.064872,1.062099,0,c_h
2,4658149,dsgdb9nsd_000004,2,3,H,-1.661639,0.0,1.0,H,1.661639,0.0,1.0,0.12211,3JHH,0.118187,3.323277,6,h_h
3,4658150,dsgdb9nsd_000004,3,0,H,1.661639,0.0,1.0,C,0.599539,0.0,1.0,0.12211,1JHC,-0.064872,1.062099,0,c_h
4,4658151,dsgdb9nsd_000004,3,1,H,1.661639,0.0,1.0,C,-0.599539,0.0,1.0,0.12211,2JHC,-0.064872,2.261178,2,c_h


,id,molecule_name,atom_index_0,atom_index_1,atom_0,x_0,y_0,z_0,atom_1,x_1,y_1,z_1,mulliken_charge_0,type,mulliken_charge_1,distance,type_enc,pairs
0,4658147,dsgdb9nsd_000004,2,0,H,-1.661639,0.0,1.0,C,0.599539,0.0,1.0,0.12211,2JHC,-0.064872,2.261178,2,c_h
1,4658148,dsgdb9nsd_000004,2,1,H,-1.661639,0.0,1.0,C,-0.599539,0.0,1.0,0.12211,1JHC,-0.064872,1.062099,0,c_h
2,4658149,dsgdb9nsd_000004,2,3,H,-1.661639,0.0,1.0,H,1.661639,0.0,1.0,0.12211,3JHH,0.118187,3.323277,6,h_h
3,4658150,dsgdb9nsd_000004,3,0,H,1.661639,0.0,1.0,C,0.599539,0.0,1.0,0.12211,1JHC,-0.064872,1.062099,0,c_h
4,4658151,dsgdb9nsd_000004,3,1,H,1.661639,0.0,1.0,C,-0.599539,0.0,1.0,0.12211,2JHC,-0.064872,2.261178,2,c_h


## Pairs Label Encode

In [0]:
pairs_dict

{'c_c': 0,
 'c_f': 1,
 'c_h': 2,
 'c_n': 3,
 'c_o': 4,
 'c_si': 5,
 'f_f': 6,
 'f_h': 7,
 'f_n': 8,
 'f_o': 9,
 'f_si': 10,
 'h_h': 11,
 'h_n': 12,
 'h_o': 13,
 'h_si': 14,
 'n_n': 15,
 'n_o': 16,
 'o_o': 17}

{'c_c': 0,
 'c_f': 1,
 'c_h': 2,
 'c_n': 3,
 'c_o': 4,
 'c_si': 5,
 'f_f': 6,
 'f_h': 7,
 'f_n': 8,
 'f_o': 9,
 'f_si': 10,
 'h_h': 11,
 'h_n': 12,
 'h_o': 13,
 'h_si': 14,
 'n_n': 15,
 'n_o': 16,
 'o_o': 17}

In [0]:
test['pairs'].unique()

array(['c_h', 'h_h', 'h_n'], dtype=object)

array(['c_h', 'h_h', 'h_n'], dtype=object)

In [0]:
test['pairs_enc']=test['pairs'].map(pairs_dict)

In [0]:
test.head()

,id,molecule_name,atom_index_0,atom_index_1,atom_0,x_0,y_0,z_0,atom_1,x_1,y_1,z_1,mulliken_charge_0,type,mulliken_charge_1,distance,type_enc,pairs,pairs_enc
0,4658147,dsgdb9nsd_000004,2,0,H,-1.661639,0.0,1.0,C,0.599539,0.0,1.0,0.12211,2JHC,-0.064872,2.261178,2,c_h,2
1,4658148,dsgdb9nsd_000004,2,1,H,-1.661639,0.0,1.0,C,-0.599539,0.0,1.0,0.12211,1JHC,-0.064872,1.062099,0,c_h,2
2,4658149,dsgdb9nsd_000004,2,3,H,-1.661639,0.0,1.0,H,1.661639,0.0,1.0,0.12211,3JHH,0.118187,3.323277,6,h_h,11
3,4658150,dsgdb9nsd_000004,3,0,H,1.661639,0.0,1.0,C,0.599539,0.0,1.0,0.12211,1JHC,-0.064872,1.062099,0,c_h,2
4,4658151,dsgdb9nsd_000004,3,1,H,1.661639,0.0,1.0,C,-0.599539,0.0,1.0,0.12211,2JHC,-0.064872,2.261178,2,c_h,2


,id,molecule_name,atom_index_0,atom_index_1,atom_0,x_0,y_0,z_0,atom_1,x_1,y_1,z_1,mulliken_charge_0,type,mulliken_charge_1,distance,type_enc,pairs,pairs_enc
0,4658147,dsgdb9nsd_000004,2,0,H,-1.661639,0.0,1.0,C,0.599539,0.0,1.0,0.12211,2JHC,-0.064872,2.261178,2,c_h,2
1,4658148,dsgdb9nsd_000004,2,1,H,-1.661639,0.0,1.0,C,-0.599539,0.0,1.0,0.12211,1JHC,-0.064872,1.062099,0,c_h,2
2,4658149,dsgdb9nsd_000004,2,3,H,-1.661639,0.0,1.0,H,1.661639,0.0,1.0,0.12211,3JHH,0.118187,3.323277,6,h_h,11
3,4658150,dsgdb9nsd_000004,3,0,H,1.661639,0.0,1.0,C,0.599539,0.0,1.0,0.12211,1JHC,-0.064872,1.062099,0,c_h,2
4,4658151,dsgdb9nsd_000004,3,1,H,1.661639,0.0,1.0,C,-0.599539,0.0,1.0,0.12211,2JHC,-0.064872,2.261178,2,c_h,2


## Connectivity

In [0]:
x=test[['pairs_enc','distance']]

In [0]:
x.head()

,pairs_enc,distance
0,2,2.261178
1,2,1.062099
2,11,3.323277
3,2,1.062099
4,2,2.261178


,pairs_enc,distance
0,2,2.261178
1,2,1.062099
2,11,3.323277
3,2,1.062099
4,2,2.261178


In [0]:
x.shape

(2505542, 2)

(2505542, 2)

In [0]:
test['bond']=model_nn.predict(x,verbose=1,batch_size=10000)

2505542/2505542 [==============================] - 49s 20us/sample


In [0]:
test.head()

,id,molecule_name,atom_index_0,atom_index_1,atom_0,x_0,y_0,z_0,atom_1,x_1,y_1,z_1,mulliken_charge_0,type,mulliken_charge_1,distance,type_enc,pairs,pairs_enc,bond
0,4658147,dsgdb9nsd_000004,2,0,H,-1.661639,0.0,1.0,C,0.599539,0.0,1.0,0.12211,2JHC,-0.064872,2.261178,2,c_h,2,0.413226
1,4658148,dsgdb9nsd_000004,2,1,H,-1.661639,0.0,1.0,C,-0.599539,0.0,1.0,0.12211,1JHC,-0.064872,1.062099,0,c_h,2,0.917924
2,4658149,dsgdb9nsd_000004,2,3,H,-1.661639,0.0,1.0,H,1.661639,0.0,1.0,0.12211,3JHH,0.118187,3.323277,6,h_h,11,0.173375
3,4658150,dsgdb9nsd_000004,3,0,H,1.661639,0.0,1.0,C,0.599539,0.0,1.0,0.12211,1JHC,-0.064872,1.062099,0,c_h,2,0.917924
4,4658151,dsgdb9nsd_000004,3,1,H,1.661639,0.0,1.0,C,-0.599539,0.0,1.0,0.12211,2JHC,-0.064872,2.261178,2,c_h,2,0.413226


,id,molecule_name,atom_index_0,atom_index_1,atom_0,x_0,y_0,z_0,atom_1,x_1,y_1,z_1,mulliken_charge_0,type,mulliken_charge_1,distance,type_enc,pairs,pairs_enc,bond
0,4658147,dsgdb9nsd_000004,2,0,H,-1.661639,0.0,1.0,C,0.599539,0.0,1.0,0.12211,2JHC,-0.064872,2.261178,2,c_h,2,0.413226
1,4658148,dsgdb9nsd_000004,2,1,H,-1.661639,0.0,1.0,C,-0.599539,0.0,1.0,0.12211,1JHC,-0.064872,1.062099,0,c_h,2,0.917924
2,4658149,dsgdb9nsd_000004,2,3,H,-1.661639,0.0,1.0,H,1.661639,0.0,1.0,0.12211,3JHH,0.118187,3.323277,6,h_h,11,0.173375
3,4658150,dsgdb9nsd_000004,3,0,H,1.661639,0.0,1.0,C,0.599539,0.0,1.0,0.12211,1JHC,-0.064872,1.062099,0,c_h,2,0.917924
4,4658151,dsgdb9nsd_000004,3,1,H,1.661639,0.0,1.0,C,-0.599539,0.0,1.0,0.12211,2JHC,-0.064872,2.261178,2,c_h,2,0.413226


In [0]:
test.shape

(2505542, 20)

(2505542, 20)

In [0]:
test['bond'].isna().value_counts()

False    2505542
Name: bond, dtype: int64

False    2505542
Name: bond, dtype: int64

In [0]:
test['bond_int']=test['bond'].apply(lambda x:int(x>0.5))

In [0]:
test.head()

,id,molecule_name,atom_index_0,atom_index_1,atom_0,x_0,y_0,z_0,atom_1,x_1,...,z_1,mulliken_charge_0,type,mulliken_charge_1,distance,type_enc,pairs,pairs_enc,bond,bond_int
0,4658147,dsgdb9nsd_000004,2,0,H,-1.661639,0.0,1.0,C,0.599539,...,1.0,0.12211,2JHC,-0.064872,2.261178,2,c_h,2,0.413226,0
1,4658148,dsgdb9nsd_000004,2,1,H,-1.661639,0.0,1.0,C,-0.599539,...,1.0,0.12211,1JHC,-0.064872,1.062099,0,c_h,2,0.917924,1
2,4658149,dsgdb9nsd_000004,2,3,H,-1.661639,0.0,1.0,H,1.661639,...,1.0,0.12211,3JHH,0.118187,3.323277,6,h_h,11,0.173375,0
3,4658150,dsgdb9nsd_000004,3,0,H,1.661639,0.0,1.0,C,0.599539,...,1.0,0.12211,1JHC,-0.064872,1.062099,0,c_h,2,0.917924,1
4,4658151,dsgdb9nsd_000004,3,1,H,1.661639,0.0,1.0,C,-0.599539,...,1.0,0.12211,2JHC,-0.064872,2.261178,2,c_h,2,0.413226,0


,id,molecule_name,atom_index_0,atom_index_1,atom_0,x_0,y_0,z_0,atom_1,x_1,...,z_1,mulliken_charge_0,type,mulliken_charge_1,distance,type_enc,pairs,pairs_enc,bond,bond_int
0,4658147,dsgdb9nsd_000004,2,0,H,-1.661639,0.0,1.0,C,0.599539,...,1.0,0.12211,2JHC,-0.064872,2.261178,2,c_h,2,0.413226,0
1,4658148,dsgdb9nsd_000004,2,1,H,-1.661639,0.0,1.0,C,-0.599539,...,1.0,0.12211,1JHC,-0.064872,1.062099,0,c_h,2,0.917924,1
2,4658149,dsgdb9nsd_000004,2,3,H,-1.661639,0.0,1.0,H,1.661639,...,1.0,0.12211,3JHH,0.118187,3.323277,6,h_h,11,0.173375,0
3,4658150,dsgdb9nsd_000004,3,0,H,1.661639,0.0,1.0,C,0.599539,...,1.0,0.12211,1JHC,-0.064872,1.062099,0,c_h,2,0.917924,1
4,4658151,dsgdb9nsd_000004,3,1,H,1.661639,0.0,1.0,C,-0.599539,...,1.0,0.12211,2JHC,-0.064872,2.261178,2,c_h,2,0.413226,0


## Total Charge

In [0]:
test['total_charge']=test['mulliken_charge_0']+test['mulliken_charge_1']

In [0]:
test.head()

,id,molecule_name,atom_index_0,atom_index_1,atom_0,x_0,y_0,z_0,atom_1,x_1,...,mulliken_charge_0,type,mulliken_charge_1,distance,type_enc,pairs,pairs_enc,bond,bond_int,total_charge
0,4658147,dsgdb9nsd_000004,2,0,H,-1.661639,0.0,1.0,C,0.599539,...,0.12211,2JHC,-0.064872,2.261178,2,c_h,2,0.413226,0,0.057238
1,4658148,dsgdb9nsd_000004,2,1,H,-1.661639,0.0,1.0,C,-0.599539,...,0.12211,1JHC,-0.064872,1.062099,0,c_h,2,0.917924,1,0.057238
2,4658149,dsgdb9nsd_000004,2,3,H,-1.661639,0.0,1.0,H,1.661639,...,0.12211,3JHH,0.118187,3.323277,6,h_h,11,0.173375,0,0.240297
3,4658150,dsgdb9nsd_000004,3,0,H,1.661639,0.0,1.0,C,0.599539,...,0.12211,1JHC,-0.064872,1.062099,0,c_h,2,0.917924,1,0.057238
4,4658151,dsgdb9nsd_000004,3,1,H,1.661639,0.0,1.0,C,-0.599539,...,0.12211,2JHC,-0.064872,2.261178,2,c_h,2,0.413226,0,0.057238


,id,molecule_name,atom_index_0,atom_index_1,atom_0,x_0,y_0,z_0,atom_1,x_1,...,mulliken_charge_0,type,mulliken_charge_1,distance,type_enc,pairs,pairs_enc,bond,bond_int,total_charge
0,4658147,dsgdb9nsd_000004,2,0,H,-1.661639,0.0,1.0,C,0.599539,...,0.12211,2JHC,-0.064872,2.261178,2,c_h,2,0.413226,0,0.057238
1,4658148,dsgdb9nsd_000004,2,1,H,-1.661639,0.0,1.0,C,-0.599539,...,0.12211,1JHC,-0.064872,1.062099,0,c_h,2,0.917924,1,0.057238
2,4658149,dsgdb9nsd_000004,2,3,H,-1.661639,0.0,1.0,H,1.661639,...,0.12211,3JHH,0.118187,3.323277,6,h_h,11,0.173375,0,0.240297
3,4658150,dsgdb9nsd_000004,3,0,H,1.661639,0.0,1.0,C,0.599539,...,0.12211,1JHC,-0.064872,1.062099,0,c_h,2,0.917924,1,0.057238
4,4658151,dsgdb9nsd_000004,3,1,H,1.661639,0.0,1.0,C,-0.599539,...,0.12211,2JHC,-0.064872,2.261178,2,c_h,2,0.413226,0,0.057238


## Dipole Moment

In [0]:
test['dp']=(-1*test['mulliken_charge_0']*test['distance']/2)+(test['mulliken_charge_1']*test['distance']/2)

In [0]:
test.head()

,id,molecule_name,atom_index_0,atom_index_1,atom_0,x_0,y_0,z_0,atom_1,x_1,...,type,mulliken_charge_1,distance,type_enc,pairs,pairs_enc,bond,bond_int,total_charge,dp
0,4658147,dsgdb9nsd_000004,2,0,H,-1.661639,0.0,1.0,C,0.599539,...,2JHC,-0.064872,2.261178,2,c_h,2,0.413226,0,0.057238,-0.211400
1,4658148,dsgdb9nsd_000004,2,1,H,-1.661639,0.0,1.0,C,-0.599539,...,1JHC,-0.064872,1.062099,0,c_h,2,0.917924,1,0.057238,-0.099297
2,4658149,dsgdb9nsd_000004,2,3,H,-1.661639,0.0,1.0,H,1.661639,...,3JHH,0.118187,3.323277,6,h_h,11,0.173375,0,0.240297,-0.006518
3,4658150,dsgdb9nsd_000004,3,0,H,1.661639,0.0,1.0,C,0.599539,...,1JHC,-0.064872,1.062099,0,c_h,2,0.917924,1,0.057238,-0.099297
4,4658151,dsgdb9nsd_000004,3,1,H,1.661639,0.0,1.0,C,-0.599539,...,2JHC,-0.064872,2.261178,2,c_h,2,0.413226,0,0.057238,-0.211400


,id,molecule_name,atom_index_0,atom_index_1,atom_0,x_0,y_0,z_0,atom_1,x_1,...,type,mulliken_charge_1,distance,type_enc,pairs,pairs_enc,bond,bond_int,total_charge,dp
0,4658147,dsgdb9nsd_000004,2,0,H,-1.661639,0.0,1.0,C,0.599539,...,2JHC,-0.064872,2.261178,2,c_h,2,0.413226,0,0.057238,-0.211400
1,4658148,dsgdb9nsd_000004,2,1,H,-1.661639,0.0,1.0,C,-0.599539,...,1JHC,-0.064872,1.062099,0,c_h,2,0.917924,1,0.057238,-0.099297
2,4658149,dsgdb9nsd_000004,2,3,H,-1.661639,0.0,1.0,H,1.661639,...,3JHH,0.118187,3.323277,6,h_h,11,0.173375,0,0.240297,-0.006518
3,4658150,dsgdb9nsd_000004,3,0,H,1.661639,0.0,1.0,C,0.599539,...,1JHC,-0.064872,1.062099,0,c_h,2,0.917924,1,0.057238,-0.099297
4,4658151,dsgdb9nsd_000004,3,1,H,1.661639,0.0,1.0,C,-0.599539,...,2JHC,-0.064872,2.261178,2,c_h,2,0.413226,0,0.057238,-0.211400


# Drop & Rearrange Columns

In [38]:
train.head()

,atom_index_0,atom_index_1,x_0,y_0,z_0,x_1,y_1,z_1,scalar_coupling_constant,distance,type_enc,atom_1_enc,atom_0_enc
0,1,0,0.002150,-0.006031,0.001976,-0.012698,1.085804,0.008001,84.8076,1.091953,0,0,1
1,1,2,0.002150,-0.006031,0.001976,1.011731,1.463751,0.000277,-11.2570,1.783120,3,1,1
2,1,3,0.002150,-0.006031,0.001976,-0.540815,1.447527,-0.876644,-11.2548,1.783147,3,1,1
3,1,4,0.002150,-0.006031,0.001976,-0.523814,1.437933,0.906397,-11.2543,1.783157,3,1,1
4,2,0,1.011731,1.463751,0.000277,-0.012698,1.085804,0.008001,84.8074,1.091952,0,0,1


In [22]:
test.head()

,id,molecule_name,atom_index_0,atom_index_1,atom_0,x_0,y_0,z_0,atom_1,x_1,y_1,z_1,mulliken_charge_0,mulliken_charge_1,type,distance,type_enc,atom_0_enc,atom_1_enc
0,4658147,dsgdb9nsd_000004,2,0,H,-1.661639,0.0,1.0,C,0.599539,0.0,1.0,NaN,NaN,2JHC,2.261178,2,1,0
1,4658148,dsgdb9nsd_000004,2,1,H,-1.661639,0.0,1.0,C,-0.599539,0.0,1.0,NaN,NaN,1JHC,1.062099,0,1,0
2,4658149,dsgdb9nsd_000004,2,3,H,-1.661639,0.0,1.0,H,1.661639,0.0,1.0,NaN,NaN,3JHH,3.323277,6,1,1
3,4658150,dsgdb9nsd_000004,3,0,H,1.661639,0.0,1.0,C,0.599539,0.0,1.0,NaN,NaN,1JHC,1.062099,0,1,0
4,4658151,dsgdb9nsd_000004,3,1,H,1.661639,0.0,1.0,C,-0.599539,0.0,1.0,NaN,NaN,2JHC,2.261178,2,1,0


In [23]:
print('train: {}'.format(train.shape))
print('test: {}'.format(test.shape))

train: (4658147, 20)
test: (2505542, 19)


In [24]:
train.dtypes

id                            int64
molecule_name                object
atom_index_0                  int64
atom_index_1                  int64
atom_0                       object
x_0                         float64
y_0                         float64
z_0                         float64
atom_1                       object
x_1                         float64
y_1                         float64
z_1                         float64
mulliken_charge_0           float64
mulliken_charge_1           float64
type                         object
scalar_coupling_constant    float64
distance                    float64
type_enc                      int64
atom_1_enc                    int64
atom_0_enc                    int64
dtype: object

In [25]:
train.columns

Index(['id', 'molecule_name', 'atom_index_0', 'atom_index_1', 'atom_0', 'x_0',
       'y_0', 'z_0', 'atom_1', 'x_1', 'y_1', 'z_1', 'mulliken_charge_0',
       'mulliken_charge_1', 'type', 'scalar_coupling_constant', 'distance',
       'type_enc', 'atom_1_enc', 'atom_0_enc'],
      dtype='object')

In [26]:
train.corr()

,id,atom_index_0,atom_index_1,x_0,y_0,z_0,x_1,y_1,z_1,mulliken_charge_0,mulliken_charge_1,scalar_coupling_constant,distance,type_enc,atom_1_enc,atom_0_enc
id,1.000000,0.200476,0.056080,-0.063553,-0.184992,-0.028264,-0.087629,-0.229034,-0.030272,-0.021828,0.003030,-0.006968,-0.019193,0.001121,0.009227,NaN
atom_index_0,0.200476,1.000000,0.145740,0.020848,-0.521941,0.078614,0.030623,-0.366010,0.021815,-0.152320,-0.041443,0.018839,0.040854,-0.028312,-0.111887,NaN
atom_index_1,0.056080,0.145740,1.000000,0.008809,-0.104602,0.005917,0.000755,-0.179888,0.056477,-0.100627,0.303508,-0.218989,0.044345,0.340554,0.497327,NaN
x_0,-0.063553,0.020848,0.008809,1.000000,0.137042,0.009599,0.637212,0.172400,0.022365,-0.012947,0.004545,-0.006015,0.001512,-0.002799,-0.000897,NaN
y_0,-0.184992,-0.521941,-0.104602,0.137042,1.000000,0.010514,0.157436,0.721484,0.018929,0.067439,-0.000514,-0.013119,-0.018573,0.010371,0.034146,NaN
z_0,-0.028264,0.078614,0.005917,0.009599,0.010514,1.000000,-0.004632,0.016645,0.540027,0.001985,-0.003237,0.004807,0.006364,-0.002951,-0.013878,NaN
x_1,-0.087629,0.030623,0.000755,0.637212,0.157436,-0.004632,1.000000,0.193026,0.011105,-0.041072,-0.005717,0.005132,0.025896,0.012117,-0.013408,NaN
y_1,-0.229034,-0.366010,-0.179888,0.172400,0.721484,0.016645,0.193026,1.000000,0.029796,0.025307,-0.061796,0.021189,-0.069445,-0.071881,-0.001055,NaN
z_1,-0.030272,0.021815,0.056477,0.022365,0.018929,0.540027,0.011105,0.029796,1.000000,-0.015298,0.010201,-0.007792,0.000562,0.001582,0.006483,NaN
mulliken_charge_0,-0.021828,-0.152320,-0.100627,-0.012947,0.067439,0.001985,-0.041072,0.025307,-0.015298,1.000000,0.018764,-0.025444,-0.041118,0.006285,0.084078,NaN


In [0]:
drop_cols=['id', 'molecule_name', 'atom_0',  'atom_1', 'type','mulliken_charge_0', 'mulliken_charge_1' ]

In [33]:
train.drop(drop_cols,axis=1,inplace=True)
train.head()

,atom_index_0,atom_index_1,x_0,y_0,z_0,x_1,y_1,z_1,scalar_coupling_constant,distance,type_enc,atom_1_enc,atom_0_enc
0,1,0,0.002150,-0.006031,0.001976,-0.012698,1.085804,0.008001,84.8076,1.091953,0,0,1
1,1,2,0.002150,-0.006031,0.001976,1.011731,1.463751,0.000277,-11.2570,1.783120,3,1,1
2,1,3,0.002150,-0.006031,0.001976,-0.540815,1.447527,-0.876644,-11.2548,1.783147,3,1,1
3,1,4,0.002150,-0.006031,0.001976,-0.523814,1.437933,0.906397,-11.2543,1.783157,3,1,1
4,2,0,1.011731,1.463751,0.000277,-0.012698,1.085804,0.008001,84.8074,1.091952,0,0,1


In [49]:
test.columns

Index(['id', 'molecule_name', 'atom_index_0', 'atom_index_1', 'atom_0', 'x_0',
       'y_0', 'z_0', 'atom_1', 'x_1', 'y_1', 'z_1', 'mulliken_charge_0',
       'mulliken_charge_1', 'type', 'distance', 'type_enc', 'atom_0_enc',
       'atom_1_enc'],
      dtype='object')

In [0]:
drop_cols=['id', 'molecule_name', 'atom_0',  'atom_1', 'type','mulliken_charge_0', 'mulliken_charge_1' ]

In [51]:
test.drop(drop_cols,axis=1,inplace=True)
test.head()

,atom_index_0,atom_index_1,x_0,y_0,z_0,x_1,y_1,z_1,distance,type_enc,atom_0_enc,atom_1_enc
0,2,0,-1.661639,0.0,1.0,0.599539,0.0,1.0,2.261178,2,1,0
1,2,1,-1.661639,0.0,1.0,-0.599539,0.0,1.0,1.062099,0,1,0
2,2,3,-1.661639,0.0,1.0,1.661639,0.0,1.0,3.323277,6,1,1
3,3,0,1.661639,0.0,1.0,0.599539,0.0,1.0,1.062099,0,1,0
4,3,1,1.661639,0.0,1.0,-0.599539,0.0,1.0,2.261178,2,1,0


In [52]:
train.isna().describe()

,atom_index_0,atom_index_1,x_0,y_0,z_0,x_1,y_1,z_1,distance,type_enc,atom_1_enc,atom_0_enc,scalar_coupling_constant
count,4658147,4658147,4658147,4658147,4658147,4658147,4658147,4658147,4658147,4658147,4658147,4658147,4658147
unique,1,1,1,1,1,1,1,1,1,1,1,1,1
top,False,False,False,False,False,False,False,False,False,False,False,False,False
freq,4658147,4658147,4658147,4658147,4658147,4658147,4658147,4658147,4658147,4658147,4658147,4658147,4658147


In [53]:
test.isna().describe()

,atom_index_0,atom_index_1,x_0,y_0,z_0,x_1,y_1,z_1,distance,type_enc,atom_0_enc,atom_1_enc
count,2505542,2505542,2505542,2505542,2505542,2505542,2505542,2505542,2505542,2505542,2505542,2505542
unique,1,1,1,1,1,1,1,1,1,1,1,1
top,False,False,False,False,False,False,False,False,False,False,False,False
freq,2505542,2505542,2505542,2505542,2505542,2505542,2505542,2505542,2505542,2505542,2505542,2505542


In [54]:
train.columns

Index(['atom_index_0', 'atom_index_1', 'x_0', 'y_0', 'z_0', 'x_1', 'y_1',
       'z_1', 'distance', 'type_enc', 'atom_1_enc', 'atom_0_enc',
       'scalar_coupling_constant'],
      dtype='object')

In [55]:
train=train[['atom_index_0', 'atom_index_1', 'x_0', 'y_0', 'z_0', 'x_1', 'y_1',
       'z_1',  'distance', 'type_enc', 'atom_1_enc',
       'atom_0_enc','scalar_coupling_constant']]
train.head()

,atom_index_0,atom_index_1,x_0,y_0,z_0,x_1,y_1,z_1,distance,type_enc,atom_1_enc,atom_0_enc,scalar_coupling_constant
0,1,0,0.002150,-0.006031,0.001976,-0.012698,1.085804,0.008001,1.091953,0,0,1,84.8076
1,1,2,0.002150,-0.006031,0.001976,1.011731,1.463751,0.000277,1.783120,3,1,1,-11.2570
2,1,3,0.002150,-0.006031,0.001976,-0.540815,1.447527,-0.876644,1.783147,3,1,1,-11.2548
3,1,4,0.002150,-0.006031,0.001976,-0.523814,1.437933,0.906397,1.783157,3,1,1,-11.2543
4,2,0,1.011731,1.463751,0.000277,-0.012698,1.085804,0.008001,1.091952,0,0,1,84.8074


In [56]:
test=test[['atom_index_0', 'atom_index_1', 'x_0', 'y_0', 'z_0', 'x_1', 'y_1',
       'z_1',  'distance', 'type_enc', 'atom_1_enc',
       'atom_0_enc']]
test.head()

,atom_index_0,atom_index_1,x_0,y_0,z_0,x_1,y_1,z_1,distance,type_enc,atom_1_enc,atom_0_enc
0,2,0,-1.661639,0.0,1.0,0.599539,0.0,1.0,2.261178,2,0,1
1,2,1,-1.661639,0.0,1.0,-0.599539,0.0,1.0,1.062099,0,0,1
2,2,3,-1.661639,0.0,1.0,1.661639,0.0,1.0,3.323277,6,1,1
3,3,0,1.661639,0.0,1.0,0.599539,0.0,1.0,1.062099,0,0,1
4,3,1,1.661639,0.0,1.0,-0.599539,0.0,1.0,2.261178,2,0,1


# Serialize Data

In [57]:
print(train.shape)
print(test.shape)

(4658147, 13)
(2505542, 12)


In [0]:
train.to_csv('train_feat_eng_v2.csv',index=False)

In [0]:
test.to_csv('test_feat_eng_v2.csv',index=False)

In [60]:
!ls

dipole_moments.csv.zip		       structures.csv.zip
kaggle.json			       structures.zip
magnetic_shielding_tensors.csv.zip     test.csv.zip
mulliken_charges.csv.zip	       test_etl.csv
potential_energy.csv.zip	       test_feat_eng_v2.csv
sample_data			       train.csv.zip
sample_submission.csv.zip	       train_etl.csv
scalar_coupling_contributions.csv.zip  train_feat_eng_v2.csv


In [61]:
df=pd.read_csv('train_feat_eng_v2.csv')
df.head()

,atom_index_0,atom_index_1,x_0,y_0,z_0,x_1,y_1,z_1,distance,type_enc,atom_1_enc,atom_0_enc,scalar_coupling_constant
0,1,0,0.002150,-0.006031,0.001976,-0.012698,1.085804,0.008001,1.091953,0,0,1,84.8076
1,1,2,0.002150,-0.006031,0.001976,1.011731,1.463751,0.000277,1.783120,3,1,1,-11.2570
2,1,3,0.002150,-0.006031,0.001976,-0.540815,1.447527,-0.876644,1.783147,3,1,1,-11.2548
3,1,4,0.002150,-0.006031,0.001976,-0.523814,1.437933,0.906397,1.783157,3,1,1,-11.2543
4,2,0,1.011731,1.463751,0.000277,-0.012698,1.085804,0.008001,1.091952,0,0,1,84.8074


In [62]:
df.shape

(4658147, 13)

In [63]:
df=pd.read_csv('test_feat_eng_v2.csv')
df.head()

,atom_index_0,atom_index_1,x_0,y_0,z_0,x_1,y_1,z_1,distance,type_enc,atom_1_enc,atom_0_enc
0,2,0,-1.661639,0.0,1.0,0.599539,0.0,1.0,2.261178,2,0,1
1,2,1,-1.661639,0.0,1.0,-0.599539,0.0,1.0,1.062099,0,0,1
2,2,3,-1.661639,0.0,1.0,1.661639,0.0,1.0,3.323277,6,1,1
3,3,0,1.661639,0.0,1.0,0.599539,0.0,1.0,1.062099,0,0,1
4,3,1,1.661639,0.0,1.0,-0.599539,0.0,1.0,2.261178,2,0,1


In [64]:
df.shape

(2505542, 12)

## Upload files to IBM Object Store

In [0]:
# The code was removed by Watson Studio for sharing.

In [0]:
from ibm_botocore.client import Config
import ibm_boto3
def upload_file_cos(credentials,local_file_name,key):  
    cos = ibm_boto3.client(service_name='s3',
    ibm_api_key_id=credentials['IBM_API_KEY_ID'],
    ibm_service_instance_id=credentials['IAM_SERVICE_ID'],
    ibm_auth_endpoint=credentials['IBM_AUTH_ENDPOINT'],
    config=Config(signature_version='oauth'),
    endpoint_url=credentials['ENDPOINT'])
    try:
        res=cos.upload_file(Filename=local_file_name, Bucket=credentials['BUCKET'],Key=key)
    except Exception as e:
        print(Exception, e)
    else:
        print('File Uploaded')

In [0]:
upload_file_cos(credentials_2,'train_feat_eng_v2_1.csv','train_feat_eng_v2_1.csv')
upload_file_cos(credentials_2,'train_feat_eng_v2_2.csv','train_feat_eng_v2_2.csv')

File Uploaded
File Uploaded


In [0]:
upload_file_cos(credentials_2,'test_feat_eng_v2.csv','test_feat_eng_v2.csv')

File Uploaded
